#### Bayesian optimise the Matern kenrel for the highest accuracy within Gaussian Process Regressor
#### Cross-validated search of hyperparameters
#### For 20 random test grid cells, each with 500 iterations

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [2]:
with open('/nfs/see-fs-02_users/earlacoa/emulator/dfs_gridcell_sample.pickle', 'rb') as ds:
    df_gridcells = pickle.load(ds)
    
df_gridcells.head()

,lat,lon,RES,IND,TRA,AGR,POW,PM2_5_DRY
0,30.5,114.25,0.23229,0.20507,0.299040,0.97374,0.55184,82.050903
1,30.5,114.25,1.12120,1.10980,0.870100,1.22220,1.25200,260.182960
2,30.5,114.25,0.79843,1.29460,0.090709,0.75401,0.19420,209.802636
3,30.5,114.25,0.34445,1.42240,0.751120,1.24090,1.21130,164.002287
4,30.5,114.25,0.62562,1.35890,1.058300,0.04688,0.64196,196.083809


In [3]:
pipe = Pipeline([
    ('scaler', Normalizer()),
    ('model', GaussianProcessRegressor(
        kernel=Matern(length_scale=1, nu=2.5),                                
        random_state=123))
])

In [8]:
search = {
    'model': Categorical([
        GaussianProcessRegressor(
            kernel=Matern(length_scale=1, nu=2.5), 
            random_state=123)
    ]),
    'model__kernel__length_scale': Real(0.1, 5.0),
    'model__kernel__nu': Real(0.5, 3.5),
    'model__normalize_y': Categorical([True, False]),
    'model__n_restarts_optimizer': Integer(1, 300)
}

In [ ]:
lats = df_gridcells[['lat', 'lon']].drop_duplicates()['lat'].values
lons = df_gridcells[['lat', 'lon']].drop_duplicates()['lon'].values

features = ['RES', 'IND', 'TRA', 'AGR', 'POW']
target = 'PM2_5_DRY'

for gridcell in df_gridcells[['lat', 'lon']].drop_duplicates().values:
    lat, lon = gridcell
    df_gridcell = df_gridcells.loc[df_gridcells.lat == lat].loc[df_gridcells.lon == lon]
    
    X = df_gridcell[features].values
    y = df_gridcell[target].values
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=123
    )
    
    loo = LeaveOneOut()
    #cv = loo.get_n_splits(X_train) # causes all NaNs at the moment, so using 5-fold for the time being
    cv = 5
    
    emulator = BayesSearchCV(
        pipe,
        [(search, 200)],
        cv=cv
    )

    emulator.fit(X_train, y_train)

    print(f"CV score: {emulator.best_score_:.4f}")
    print(f"test score: {emulator.score(X_test, y_test):.4f}")

/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_ite

CV score: -40.1666
test score: -2.5243


/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_ite

CV score: -12.3973
test score: -0.1018


/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/nfs/a68/earlacoa/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_ite